In [1]:
import pandas as pd

In [24]:
df_rodent = pd.read_csv('Rodent_Inspection_20240710.csv', nrows=2)

In [28]:
df_rodent.head()

,INSPECTION_TYPE,JOB_ID,BOROUGH,INSPECTION_DATE,RESULT
0,Initial,PC8122655,Manhattan,06/07/2023 04:36:13 PM,Passed
1,Initial,PC7504205,Staten Island,09/16/2010 04:00:00 PM,Failed for Other R
2,Initial,PC7744572,Bronx,07/28/2021 12:48:53 PM,Passed
3,Initial,PC7732344,Bronx,07/07/2021 08:49:45 AM,Passed
4,Initial,PC7906420,Bronx,07/19/2022 01:50:56 PM,Passed


In [53]:
df_restaurant = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results_20240710.csv',
                            usecols=['CAMIS', 'BORO', 'ZIPCODE', 'INSPECTION DATE', 'VIOLATION CODE', 'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE', 'GRADE DATE', 'INSPECTION TYPE'],
                            dtype={
                                'CAMIS' : str,
                                'BORO' : 'category',
                                'ZIPCODE' : 'category',
                                'INSPECTION DATE' : object,
                                'VIOLATION CODE' : 'category',
                                'VIOLATION DESCRIPTION' : str,
                                'CRITICAL FLAG': 'category',
                                'SCORE': float,
                                'GRADE' : 'category',
                                'GRADE DATE': object,
                                'INSPECTION TYPE': 'category'
                            })
df_restaurant['INSPECTION DATE'] = pd.to_datetime(df_restaurant['INSPECTION DATE'])
df_restaurant['INSPECTION GRADE DATE'] = pd.to_datetime(df_restaurant['GRADE DATE'])


In [63]:
df_rodent = pd.read_csv('Rodent_Inspection_20240710.csv',
                        usecols=['INSPECTION_TYPE', 'JOB_ID', 'BOROUGH', 'INSPECTION_DATE', 'RESULT', 'ZIP_CODE'],
                        dtype={
                            'INSPECTION_TYPE' : 'category',
                            'JOB_ID' : str,
                            'BOROUGH' : 'category',
                            'INSPECTION_DATE' : object,
                            'RESULT' : 'category',
                            'INSPECTION_DATE':object,
                            'ZIP_CODE' : 'category'
                        },
)
df_rodent['INSPECTION_DATE'] = pd.to_datetime(df_rodent['INSPECTION_DATE']
                                              , errors='coerce'
                                              , format='%m/%d/%Y %I:%M:%S %p')

## Clean Data

### Clean Restaurant Data

In [ ]:
df_restaurant.info()

In [54]:
df_restaurant = df_restaurant[(df_restaurant['INSPECTION DATE'].dt.year == 2019) 
                              & (df_restaurant['ZIPCODE'] not in ['12345', '08512'])
                              & (df_restaurant['ZIPCODE'].notna())
                              & (df_restaurant['SCORE'].notna()
                                 )]

In [57]:
df_restaurant.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2219 entries, 721 to 237205
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CAMIS                  2219 non-null   object        
 1   BORO                   2219 non-null   category      
 2   ZIPCODE                2219 non-null   category      
 3   INSPECTION DATE        2219 non-null   datetime64[ns]
 4   VIOLATION CODE         2194 non-null   category      
 5   VIOLATION DESCRIPTION  2194 non-null   object        
 6   CRITICAL FLAG          2219 non-null   category      
 7   SCORE                  2219 non-null   float64       
 8   GRADE                  1283 non-null   category      
 9   GRADE DATE             1255 non-null   object        
 10  INSPECTION TYPE        2219 non-null   category      
 11  INSPECTION GRADE DATE  1255 non-null   datetime64[ns]
dtypes: category(6), datetime64[ns](2), float64(1), object(3)
memory

### Group the restaurant scores by zipcode and take the mean

In [59]:
df_restaurant_group = df_restaurant.groupby(by=['ZIPCODE']
                                            , dropna=False
                                            , observed=False
                                            , as_index=False).agg({
                                                'SCORE' : 'mean'
                                            })

In [60]:
df_restaurant_group

,ZIPCODE,SCORE
0,08512,NaN
1,10000,NaN
2,10001,13.055556
3,10002,15.096774
4,10003,19.687500
...,...,...
221,10115,NaN
222,10279,NaN
223,11550,NaN
224,14057,NaN


### Clean Rodent Data

In [65]:
df_rodent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2584965 entries, 0 to 2584964
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   INSPECTION_TYPE  category      
 1   JOB_ID           object        
 2   ZIP_CODE         category      
 3   BOROUGH          category      
 4   INSPECTION_DATE  datetime64[ns]
 5   RESULT           category      
dtypes: category(4), datetime64[ns](1), object(1)
memory usage: 51.8+ MB


In [74]:
df_rodent = df_rodent[(df_rodent['INSPECTION_DATE'].dt.year == 2019) 
                              & ~(df_rodent['ZIP_CODE'].isin(['0','10000','12345', '08512']))
                              & (df_rodent['ZIP_CODE'].notna())
                              ]

In [75]:
df_rodent.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242747 entries, 7 to 2584959
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   INSPECTION_TYPE  242747 non-null  category      
 1   JOB_ID           242747 non-null  object        
 2   ZIP_CODE         242747 non-null  category      
 3   BOROUGH          242747 non-null  category      
 4   INSPECTION_DATE  242747 non-null  datetime64[ns]
 5   RESULT           242747 non-null  category      
dtypes: category(4), datetime64[ns](1), object(1)
memory usage: 6.7+ MB


In [76]:
df_rodent['RESULT'].unique()

['Passed', 'Bait applied', 'Rat Activity', 'Failed for Other R']
Categories (5, object): ['Bait applied', 'Failed for Other R', 'Monitoring visit', 'Passed', 'Rat Activity']

In [78]:
df_rodent_grouped = df_rodent.groupby(by='ZIP_CODE'
                                      , dropna=False
                                      , observed=False
                                      , as_index=False)['RESULT'].agg(
                                          lambda x : (x == 'Rat Activity').sum()
                                      ).reset_index()
df_rodent_grouped.rename(columns = {'RESULT' : 'count_rat_activity'}, inplace=True)

In [79]:
df_rodent_grouped.head()

,index,ZIP_CODE,count_rat_activity
0,0,0,0
1,1,10001,89
2,2,10002,938
3,3,10003,446
4,4,10004,15


## Merge the 2 grouped datasets

In [ ]:
df_merged = df_rodent_grouped.merge(left_on=['ZIP_CODE'],
                                    right_on='ZIPCODE')